In [1]:
from ipywidgets import widgets,Layout
from IPython.core.display import display, HTML
from IPython.display import clear_output

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os
# fix random seed for reproducibility
from numpy.random import seed
seed(101)
from tensorflow import set_random_seed
set_random_seed(101)

import pandas as pd
import numpy as np

import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import binary_accuracy

import os
import cv2
import glob
import imageio
import skimage
import skimage.io
import skimage.transform

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline
import os, sys 
from tkinter import filedialog
from shutil import copyfile, copy
import shutil

Using TensorFlow backend.


In [3]:
l = Layout(flex='0 1 auto', height='300px', min_height='40px', width='auto')
score_output2 = widgets.Output(value='',layout=l)

l = Layout(flex='0 1 auto', height='20px', min_height='40px', width='auto')
score_output1 = widgets.Output(value='',layout=l)

cwd = os.getcwd()

In [4]:
def choose_file():
    global cwd
    path = filedialog.askopenfilename()
    copy(path, '/'.join([cwd,"base_dir/test/Tuberculosis/"]))
    return 0

In [5]:
model = None
NUM_AUG_IMAGES_WANTED = 1000 
IMAGE_HEIGHT = 96
IMAGE_WIDTH = 96


kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.3

if(model==None):
    model = Sequential()
    model.add(Conv2D(first_filters, kernel_size, activation = 'relu', 
                     input_shape = (IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
    model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
    model.add(MaxPooling2D(pool_size = pool_size)) 
    model.add(Dropout(dropout_conv))

    model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
    model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
    model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
    model.add(MaxPooling2D(pool_size = pool_size))
    model.add(Dropout(dropout_conv))

    model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
    model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
    model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
    model.add(MaxPooling2D(pool_size = pool_size))
    model.add(Dropout(dropout_conv))

    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(dropout_dense))
    model.add(Dense(2, activation = "softmax"))
    model.load_weights("model.h5")
    
def tb_analyzer():
    global model
    global cwd
    valid_path = 'base_dir/test'
    num_val_samples = 1
    val_batch_size = 1
    val_steps = np.ceil(num_val_samples / val_batch_size)
    datagen = ImageDataGenerator(rescale=1.0/255)
    test_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_HEIGHT,IMAGE_WIDTH),
                                        batch_size=val_batch_size,
                                        class_mode='categorical',
                                        shuffle=False)
    predictions = model.predict_generator(test_gen, steps=val_steps, verbose=2)
    files = glob.glob('/'.join([cwd,'base_dir/test/Tuberculosis/*']))
    for f in files:
        with score_output2:
            img = cv2.imread(f)
            plt.imshow(img)
            plt.show()
        os.remove(f)
        
    if(predictions[0][1]>0.5):
        return 'Person has TB'
    else:
        return 'Person dont have TB'
    
    
def sender(sender):
    score_output1.clear_output()
    score_output2.clear_output()
    with score_output1:
        print("Getting answer . . . .")
    file = choose_file()
    pred = tb_analyzer()
    score_output1.clear_output()
    with score_output1:
        print(pred)

In [6]:
button1 = widgets.Button(description='Select X-Ray',layout=Layout(width='50%',height='30px'))
button1.style.button_color = '#2196f3'
display(button1)
button1.on_click(sender)

Button(description='Select X-Ray', layout=Layout(height='30px', width='50%'), style=ButtonStyle(button_color='…

Found 1 images belonging to 1 classes.


In [7]:
score_output2

Output(layout=Layout(flex='0 1 auto', height='200px', min_height='40px', width='auto'))

In [8]:
score_output1

Output(layout=Layout(flex='0 1 auto', height='20px', min_height='40px', width='auto'))

In [9]:
# # json_file = open('./model_tb_1/model.json', 'r')
# # loaded_model_json = json_file.read()
# # json_file.close()
# # loaded_model = model_from_json(loaded_model_json)
# # # load weights into new model
# model.load_weights("model.h5")
# print("Loaded model from disk")

In [10]:
# train_path = 'base_dir/train_dir'
# # valid_path = 'base_dir/val_dir'
# valid_path = 'base_dir/test'

# # num_train_samples = len(df_train)
# num_val_samples = 4#len(df_val)
# train_batch_size = 10
# val_batch_size = 4


# # train_steps = np.ceil(num_train_samples / train_batch_size)
# val_steps = np.ceil(num_val_samples / val_batch_size)

# datagen = ImageDataGenerator(rescale=1.0/255)

# # valid_path = '/home/shabaz/Documents/resume/Tuberculosis-TB-Analyzer/base_dir/test'
# test_gen = datagen.flow_from_directory(valid_path,
#                                         target_size=(IMAGE_HEIGHT,IMAGE_WIDTH),
#                                         batch_size=val_batch_size,
#                                         class_mode='categorical',
#                                         shuffle=False)

In [11]:
# predictions = model.predict_generator(test_gen, steps=val_steps, verbose=2)

In [12]:
# if(predictions[2][1]>0.5):
#     print('person has tuberculosis')
# else:
#     print('person does not have tuberculosis')

In [13]:
# test_gen.filenames 

In [14]:
# shutil.rmtree('/home/shabaz/Documents/resume/Tuberculosis-TB-Analyzer/base_dir/test/Tuberculosis/*')

In [15]:
# shutil.rmtree('/home/shabaz/Documents/resume/Tuberculosis-TB-Analyzer/base_dir/test/Tuberculosis')

In [19]:
# cwd

In [18]:
display(HTML(r"""<style id=hide>div.input{display:none;}</style><button type="button"onclick="var myStyle = document.getElementById('hide').sheet;myStyle.insertRule('div.input{display:inherit !important;}', 0);"> </button>"""))